In [1]:
from typing import NamedTuple
from collections import defaultdict
from dataclasses import dataclass

import matplotlib.pyplot as plt
import numpy as np
import utils

## Day 8: Seven Segment Search

[#](https://adventofcode.com/2021/day/8) We have 7 segment (a-g) displays showing us a digit, figure out which pattern is what digit (0-9).

* 1 uses 2 segements
* 4 is the only digit using 4 segments
* 7 is the only digit using 3 segments
* 8 uses all 7 segments
* the digits besides 1,4,7 don't use unique num of segments

the non unique segments are:

* 2,3,5: 5 segments
* 0,6,9: 6 segments

the digits look like:

```
  0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg
```

The inputs are 10 strings giving us the segements for each unique digit, than after the ` | ` sepertor 4 strings giving us the output.

In [2]:
test: str = """acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf
"""

test2: str = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

raw_inp = utils.get_input(8, splitlines=False)

def parse(inp=test, verbose=False):
    """takes in the raw input and returns a list of tuples
        ([segments x 10], [output x 4])"""
    data = []
    for line in inp.splitlines():
        digits, output = line.split(" | ")
        # sorting them since of course they will come in jumbled
        digits = ["".join(sorted(d)) for d in digits.split(" ")]
        output = ["".join(sorted(d)) for d in output.split(" ")]

        data.append((digits, output))
    return data

print(parse())

[(['abcdefg', 'bcdef', 'acdfg', 'abcdf', 'abd', 'abcdef', 'bcdefg', 'abef', 'abcdeg', 'ab'], ['bcdef', 'abcdf', 'bcdef', 'abcdf'])]


Part 1 is easy - just loop through all the outputs annd count the ones we're looking for. So I'm not going to bother figuring out anything for part 1 as we can just count the outputs instead for all the unique digits:

In [3]:
def solve(inp=test, verbose: bool = False):
    """counts num of unique segments in output"""
    data = parse(inp)

    segs_seen = defaultdict(int) # holding segments seen

    for digits, output in data:
        for n in output:
            segs_seen[len(n)] += 1

    if verbose: print(segs_seen)

    return sum([segs_seen[num] for num in (2,4,3,7)])


assert solve(test2) == 26
solve(raw_inp)

534

## Part 2

So before we didn't have to figure out how exactly the wires are crossed. So now we need to to that to figure out all the digits, not just the ones with using unique num of segments.

There would be a mathematical way to do this, but since we're only dealing with 10 digits I'm going with the stare at the problem till it makes sense approach.

Listing the num of segments:

* 2 segments: 1
* 3 segments: 7
* 4 segments: 4
* 5 segments: 2,3,5
* 6 segments: 0,6,9
* 7 segments: 8

For the 5 segment digits, lets see how they overlap with the 4 segment unique number 4.

* the number 2 has 3 segments in commonn with 4, 3 and 5 have 3. 

In [13]:
def codebreak(segments, verbose=False):
    """takes in string of ten digits and returns a dict of string to num"""
    str_to_num = {} # translates string to digit

    #first handle 1,4,7,8 nums which have unique number of segments
    for digit in segments:
        match (n := len(digit)):
            case 2:
                str_to_num[digit] = 1
            case 3:
                str_to_num[digit] = 7
            case 4:
                str_to_num[digit] = 4
            case 7:
                str_to_num[digit] = 8

    num_to_str = {v: k for k, v in str_to_num.items()}

    # deal with five segment numbers
    five_segments = [digit for digit in segments if len(digit)==5]

    # num 2 has two segments in common with with 4
    two = [d for d in five_segments if len(set(num_to_str[4]) & set(d))==2][0]

    # num 3 has 4 segments in common with 2
    three = [d for d in five_segments if len(set(two) & set(d))==4][0]

    # num 5 has 3 segments in common with 2
    five = [d for d in five_segments if len(set(two) & set(d))==3][0]

    # six segment numbers 0,6,9
    six_segments = [digit for digit in segments if len(digit)==6]

    # num 6 out of 0,6,9 has only 1 digit in common with 1
    six = [d for d in six_segments if len(set(num_to_str[1]) & set(d))==1][0]

    # we have now only 0 and 9 left
    # the number 0 overlaps 4 times with 9 and 3 times with zero
    nine = [d for d in six_segments if len(set(num_to_str[4]) & set(d))==4][0]
    #zero = [d for d in six_segments if len(set(num_to_str[4]) & set(d))==3][0]
    # now zero is just the last remaining six segment
    zero = (set((six,nine)) ^ set(six_segments)).pop()

    # add these new vals to our dicts
    for k, v in zip((zero, two, three, five, six, nine), (0, 2, 3, 5, 6, 9)):
        str_to_num[k] = v
        num_to_str[v] = k

    assert len(str_to_num) == 10, "Don't have ten keys in str_to_num" 

    # print the status
    if verbose:
        print(f"{len(num_to_str)} keys figured out")
        for n in range(10):
            try:
                print(f"{n:2}: {num_to_str[n]:7}")
            except:
                pass

    return str_to_num

data = parse(test)
segments, out = data[0]
str_to_num = codebreak(segments, True)
print(str_to_num)


10 keys figured out
 0: abcdeg 
 1: ab     
 2: acdfg  
 3: abcdf  
 4: abef   
 5: bcdef  
 6: bcdefg 
 7: abd    
 8: abcdefg
 9: abcdef 
{'abcdefg': 8, 'abd': 7, 'abef': 4, 'ab': 1, 'abcdeg': 0, 'acdfg': 2, 'abcdf': 3, 'bcdef': 5, 'bcdefg': 6, 'abcdef': 9}


In [12]:
def solve_2(inp=test, total:int=0, verbose: bool = False):
    data = parse(inp)
    for row in data:
        segments, out = row
        str_to_num = codebreak(segments)
        num = int("".join([str(str_to_num[n]) for n in out]))
        total += num

    return total


assert solve_2(test2) == 61229 # example answer
solve_2(raw_inp)

1070188

This was an interesting one - initially I looked at and thought I would have to eyeball the patterns to figure out what they meant, but simple logic by looking at the difference in segments was enough.

This solution only works for a 7 segment display, a more clever one should be able to scale up or down, but that requires some maths.

## Display

for future visualization

In [133]:
@dataclass
class Display:
    inp: str = ""
    x, y = 6, 7
    
    def make_display(self):
        self.digits = [self.make_digit() for _ in range(4)]

    self.make_display(self)

    def make_digit(self):
        """makes a blank grid to hold a digit"""
        digit = np.full((y,x), ".")
        # blank the mid bits
        digit[1:3,1:5] = " "
        digit[4:6,1:5] = " "
        return digit
        
    
    def turn_on(self, segment:str, digit:int=0):
        match segment:
            case "a":
                self.digits[digit][0, 1:5] = "a"
            case "b":
                self.digits[digit][1:3, 0] = "b"
            case "c":
                self.digits[digit][1:3, 5] = "c"
            case "d":
                self.digits[digit][3, 1:5] = "d"
            case "e":
                self.digits[digit][4:6, 0] = "e"
            case "f":
                self.digits[digit][4:6, 5] = "f"
            case "g":
                self.digits[digit][6, 1:5] = "g"
                
    
    def show(self, inp: str, digit=0):
        #self.reset()
        self.inp = inp
        for segment in inp:
            self.turn_on(segment, digit)
            
    def see(self, ax=None):
        """shows as an image or returns it as an ax obj"""
        [print("\n".join(
            ["".join(row) for row in digit] for digit in self.digits))]
                    
                
    
d = Display()

d.show("abcdefg", 0)
d.see()

NameError: name 'self' is not defined